In [ ]:
import requests
import sqlite3
from google.transit import gtfs_realtime_pb2

In [47]:
def parse_feed(feed):
    vehicle_positions = []
    for entity in feed.entity:
        if entity.HasField('vehicle'):
            vehicle = entity.vehicle
            # print("occupancy is ", str(vehicle.occupancy_status))
            vehicle_positions.append({
                'id': entity.id,
                'vehicle_id': vehicle.vehicle.id,
                'trip_id': vehicle.trip.trip_id,
                'route_id': vehicle.trip.route_id,
                'latitude': vehicle.position.latitude,
                'longitude': vehicle.position.longitude,
                'bearing': vehicle.position.bearing,
                'speed': vehicle.position.speed,
                'timestamp': vehicle.timestamp,
                'occupancy_status': str(vehicle.occupancy_status)
            })
    return vehicle_positions

In [48]:
def insert_vehicle_positions_table(cursor, data):
    cursor.execute('''
    INSERT INTO vehicle_positions (
    id, vehicle_id, trip_id, route_id, timestamp,
    latitude, longitude, bearing, speed, occupancy_status
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (
        data['id'], data['vehicle_id'], data['trip_id'], 
        data['route_id'], data['timestamp'], data['latitude'], 
        data['longitude'], data['bearing'], data['speed'], 
        data['occupancy_status']
    ))

In [49]:
# Parse realtime protobuf feed
feed = gtfs_realtime_pb2.FeedMessage()
response = requests.get('https://bustime.ttc.ca/gtfsrt/vehicles')
feed.ParseFromString(response.content)

vehicle_positions = parse_feed(feed)
print(len(vehicle_positions))

1515


In [50]:
conn = sqlite3.connect('../rtds-ttc.db')
cursor = conn.cursor()

for record in vehicle_positions:
    insert_vehicle_positions_table(cursor, record)

conn.commit()
conn.close()


In [51]:
conn.commit()
conn.close()

ProgrammingError: Cannot operate on a closed database.